# Reversal with age

### Load libraries

In [1]:
%config Completer.use_jedi = False

import pandas as pd
import numpy as np
import scipy, os
from glob import glob

import matplotlib.pyplot as plt
import seaborn as sns

import theano
import theano.tensor as tt

import update_q # this is helper functions I used as a seperate file to make the notebook more readable

import pymc3 as pm
import arviz as az

### Load the data

For the data you need the demographics file, created using a different script and access to the S@Y.

In [2]:
age = pd.read_csv('../demo.csv')
glober = '/media/Data/Lab_Projects/Aging/behavioral/Reversal/AG_*_RV/ETLearning_*.csv' # file location on S@Y
db = pd.DataFrame()

for sub in glob(glober):
    
    try:
        df = pd.read_csv(sub)
        df['sub'] = sub.split('_')[2]
        if df.shape[0] == 70:
            db = pd.concat([db, df], axis = 0)
            #db = db.append(df)#[df.trialNum<36]) # to be used in for ACQ analysis only
    except:
        print(sub)
        print('error')

db = db.sort_values(by=['sub','trialNum'])

print('number of subject: ', len(db['sub'].unique()))

number of subject:  98


#### Clean data

Remove subject with low MoCA (early dementia)

In [3]:
db['sub'] = db['sub'].astype('int')
db = db.merge(age, left_on='sub', right_on='sub')
db = db.sort_values('sub')
print('Valid subjects: ', len(np.unique(db['sub'])))

Valid subjects:  71


Remove subject who didn't respond in time for more than 5 trials

In [4]:
x = db[db['RT']==-1].groupby('sub').count()['rating']
x = x[x>5].reset_index()
print(x)
db = db[~db['sub'].isin(x['sub'].values)]

   sub  rating
0  102      13


### Data preparation

extract number of subjects and indexes for modeling 

In [6]:
n_subj   = len(db['sub'].unique())
n_trials = max(db.trialNum) # all have 70 trials

trials, subj = np.meshgrid(list(range(n_trials)), range(n_subj))
trial = (trials<35)*1 # for modeling alpha for ACQ and reversal phases
trials = tt.as_tensor_variable(trials.T)
trial = tt.as_tensor_variable(trial.T)
subj   = tt.as_tensor_variable(subj.T)


stim   = np.reshape([db['rectOri']],   (n_subj, n_trials)).T # tilt of the X (0 or 45)
reward = np.reshape([db['rectValue']], (n_subj, n_trials)).T # was there a reward (0 or 6)
rating = np.reshape([db['rating']],    (n_subj, n_trials)).T # response 1-9
rating = [np.nan if 0 else x/9 for x in rating]

stim   = np.array(stim/45,  dtype='int')
reward = np.array(reward/6, dtype='int')

stim = tt.as_tensor_variable(stim)
reward = tt.as_tensor_variable(reward)

Age can be entered as an int 18-88 or as a float .18-.88.

Using int will be more similar to the stimulus and reward 

However I think the float will be easier to fit as it will be on the same scale.

In [8]:
ageT = np.reshape([db['age']],   (n_subj, n_trials)).T
# ageT = np.array(ageT,  dtype='int')
ageT = np.array(ageT/100,  dtype='float')
ageT = tt.as_tensor_variable(ageT)

In [37]:
ageList = db[['sub','age']].drop_duplicates()
ageList = ageList['age'].values/100


array([0.18, 0.43, 0.48, 0.26, 0.58, 0.74, 0.59, 0.83, 0.32, 0.56, 0.33,
       0.27, 0.28, 0.67, 0.7 , 0.79, 0.72, 0.65, 0.27, 0.23, 0.22, 0.67,
       0.52, 0.79, 0.59, 0.81, 0.24, 0.74, 0.74, 0.22, 0.21, 0.74, 0.43,
       0.67, 0.74, 0.54, 0.54, 0.37, 0.29, 0.57, 0.7 , 0.64, 0.18, 0.7 ,
       0.7 , 0.24, 0.26, 0.88, 0.79, 0.65, 0.5 , 0.71, 0.57, 0.86, 0.61,
       0.43, 0.83, 0.75, 0.19, 0.29, 0.19, 0.21, 0.73, 0.21, 0.35, 0.19,
       0.26, 0.33, 0.21, 0.3 ])

### Tuning parameters

In [9]:
tune = 2000 # 2000
draws = 2000
target_accept = .95 #.99

### Modeling 

#### Alpha calculated before updating

In [ ]:
with pm.Model() as model_RW_Pyage:
    
    # connecting function
    mu = pm.Normal('mu', 0, 1)
    sd = pm.HalfNormal('sd',2) 
    intercept = pm.Normal('intercept', mu, sd, shape=n_subj)
    
    beta_h = pm.Normal('beta_h', 0,1)
    beta_sd = pm.HalfNormal('beta_sd', 1)
    beta = pm.Normal('beta',beta_h, beta_sd, shape=n_subj)
    
    # Age slope
    Age = pm.Normal('Age', 0, 1)
        
    
    # Model Hyper priors
    a_a = pm.TruncatedNormal('a_a', 3, 2, lower=1)
    a_b = pm.TruncatedNormal('a_b', 7, 2, lower=1)
    eps = pm.HalfNormal('eps', 5)
    
    
    alpha = pm.Beta('alpha', a_a, a_b, shape=n_subj)
    
    alphaAge = alpha + Age*ageList
    
    
    
    Qs = .5 * tt.ones((n_subj,2), dtype='float64') # set values for boths stimuli (CS+, CS-)
    vec = .5 * tt.ones((n_subj,1), dtype='float64') # vector to save the relevant stimulus's expactation
    
    [Qs,vec], updates = theano.scan(
        fn=update_q.update_Q,
        sequences=[stim, reward],
        outputs_info=[Qs, vec],
        non_sequences=[alphaAge, n_subj])
   
    
    vec_ = vec[trials,subj,0] * beta[subj] + intercept[subj]
    
    learn = pm.Normal('learn', vec_, eps, observed=rating) 
    
    # add matrix of expected values (trials X subjects)
    ev = pm.Deterministic('expected_value', vec_)
    
    trace_RW_h_age = pm.sample(tune=tune, draws=draws, return_inferencedata=True, target_accept= target_accept)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, eps, a_b, a_a, Age, beta, beta_sd, beta_h, intercept, sd, mu]


#### Alpha calculated as part of the updating process

In [ ]:
with pm.Model() as model_RW_Pyage:
    
    # connecting function
    mu = pm.Normal('mu', 0, 1)
    sd = pm.HalfNormal('sd',2) 
    intercept = pm.Normal('intercept', mu, sd, shape=n_subj)
    
    beta_h = pm.Normal('beta_h', 0,1)
    beta_sd = pm.HalfNormal('beta_sd', 1)
    beta = pm.Normal('beta',beta_h, beta_sd, shape=n_subj)
    
    # Age slope
    Age = pm.Normal('Age', 0, 1)
        
    
    # Model Hyper priors
    a_a = pm.TruncatedNormal('a_a', 3, 2, lower=1)
    a_b = pm.TruncatedNormal('a_b', 7, 2, lower=1)
    eps = pm.HalfNormal('eps', 5)
    
    alpha = pm.Beta('alpha', a_a, a_b, shape=n_subj)

    
    Qs = .5 * tt.ones((n_subj,2), dtype='float64') # set values for boths stimuli (CS+, CS-)
    vec = .5 * tt.ones((n_subj,1), dtype='float64') # vector to save the relevant stimulus's expactation
    
    [Qs,vec], updates = theano.scan(
        fn=update_q.update_Q_age,
        sequences=[stim, reward, ageT],
        outputs_info=[Qs, vec],
        non_sequences=[alpha, n_subj])
   
    
    vec_ = vec[trials,subj,0] * beta[subj] + intercept[subj]
    
    learn = pm.Normal('learn', vec_, eps, observed=rating) 
    
    # add matrix of expected values (trials X subjects)
    ev = pm.Deterministic('expected_value', vec_)
    
    trace_RW_h_age = pm.sample(tune=tune, draws=draws, return_inferencedata=True, target_accept= target_accept)